<a href="https://colab.research.google.com/github/bsaha205/Spring_23_MLO/blob/main/HW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 1

$S_α(\mathbf{x_0}) = argmin_{x \in R^d}\frac{1}{2α} \|\mathbf{x} - \mathbf{x_0}\|^2 + \|x\|_1$

To show that the soft-shrinkage operator on vectors given by $S_α(x_0)$ is well defined, we need to show that the objective function is strictly convex.
Now, the first term is a strictly convex function as it's hessian metrix is positive definite. And the second term is also convex since the L1-norm n is convex. Therefore, the sum of the two terms is stricly convex function that means the objective function is well difined and there is a unique minimizer.


Now, from the participation 4 for a scalar input a, we have,

$s_α(a) = \begin{cases}
a - \alpha, & \text{if } a > \alpha \\
0, & \text{if } -\alpha \leq a \leq \alpha \\
a + \alpha, & \text{if } a < -\alpha
\end{cases}$


Now we have,

$S_α(\mathbf{x_0}) = argmin_{x \in R^d}\frac{1}{2α} \|\mathbf{x} - \mathbf{x_0}\|^2 + \|x\|_1 $

$S_α(\mathbf{x_0}) = argmin_{x \in R^d}\frac{1}{2α} \sum_{i=1}^d (x_i - (x_0)_i)^2 + \sum_{i=1}^d |x_i| $

$S_α(\mathbf{x_0}) = argmin_{x \in R^d} \sum_{i=1}^d \frac{1}{2α}[(x_i - (x_0)_i)^2 + |x_i|] $ 

Now after seperating the individual component we can write,

$S_α(\mathbf{x_0})_i  = s_α((\mathbf{x_0})_i)$ 

After simplyfying $s_α((\mathbf{x_0})_i)$  we can write,

$ s_α((\mathbf{x_0})_i) = sign((\mathbf{x_0})_i) max(|(\mathbf{x_0})_i| -α, 0)$



The implementation of **softShrink(x0, alpha)** is given below.

In [3]:
import numpy as np

def softShrink(x0, alpha):
    return np.sign(x0) * np.maximum(np.abs(x0) - alpha, 0)

# Problem 3

We have the LASSO problem that is,

$\mathbf{x}^* = argmin_{x \in R^d}\frac{1}{2} \|\mathbf{Ax} - b\|_2^2 + λ\|x\|_1$

Assume, $f(\mathbf{x}) = \frac{1}{2} \|\mathbf{Ax} - b\|_2^2$ and $g(\mathbf{x}) = λ\|x\|_1$,

So we write the above LASSO problem in the form of $min_{\mathbf{x}} f(\mathbf{x})+ g(\mathbf{x})$

Now from the (Prox) of Problem-2 we can write that,

$\mathbf{x}_{t+1} = argmin_{x \in R^d}\frac{1}{2α_t} \|\mathbf{x} - (\mathbf{x}_t - α_t\nabla f(\mathbf{x}_t) \|_2^2 + g(\mathbf{x})$

$\mathbf{x}_{t+1} = argmin_{x \in R^d}\frac{1}{2α_t} \|\mathbf{x} - (\mathbf{x}_t - α_t \mathbf{A}^T (\mathbf{Ax_t} - b)\|_2^2 +  λ\|x\|_1$

$\mathbf{x}_{t+1} = s_{αλ} (\mathbf{x}_t - α_t \mathbf{A}^T (\mathbf{Ax_t} - b))$

Where $s_{αλ}$ is the soft-shrinkage operator and defined as,

$ s_α(\mathbf{y}) = sign(\mathbf{y}) * max(|\mathbf{y}| - αλ, 0)$


The implementation of the given function [xT, objhist] = istaLasso(A, b, lambda_reg, x0, alpha, T) is given below.

In [ ]:
import numpy as np

def istaLasso(A, b, lambda_reg, x0, alpha, T):
    
    # Initialize variables
    x_t = x0
    objhist = []

    # appending the initial objective function
    objhist.append(np.linalg.norm(A @ x_t - b) ** 2 / 2 + lambda_reg * np.linalg.norm(x_t, ord=1))
    
    for i in range(T):
        # Compute y as mentioned
        y = x_t - alpha (A.T @ (A @ x_t - b))
        
        # Update estimate of x
        x_t = softShrink(y, alpha*lambda_reg)
        
        # Compute objective value at current estimate of x
        objhist.append(np.linalg.norm(A @ x_t - b) ** 2 / 2 + lambda_reg * np.linalg.norm(x_t, ord=1))
    
    return x_t, objhist

# Problem 4

The LASSO problem is,

$\mathbf{x}^* = argmin_{x \in R^d}\frac{1}{2} \|\mathbf{Ax} - b\|_2^2 + λ\|\mathbf{x}\|_1$

Now, the gradient of the first part is straight forward which is, $\mathbf{A}^T (\mathbf{Ax} - b)$.

And the gradient of the second part can be witten as $λs$ where,

$s_i = \begin{cases} \text{sign}(x_i), & x_i \neq 0\\ [-1,1], & x_i = 0 \end{cases}$



The implementaion of the  g = lassoSubgrad(A, b, lambda_reg, x) function that returns a subgradient for the LASSO objective at x is given below.

In [ ]:
def lassoSubgrad(A, b, lambda_reg, x):
    # compute s 
    s = np.zeros_like(x)
    s[x > 0] = 1
    s[x < 0] = -1
    s[np.abs(x) <= 1e-8] = np.random.uniform(low=-1, high=1, size=s[np.abs(x) <= 1e-8].shape)
    
    return A.T @ (A @ x - b) + lambda_reg * s

The implementaion of the [xT, objhist] = subgradLasso(A, b, lambda_reg, x0, alpha, T) is given below.

In [ ]:
import numpy as np

def subgradLasso(A, b, lambda_reg, x0, alpha, T):
    
    # Initialize variables
    x_t = x0
    objhist = []

    # appending the initial objective function
    objhist.append(np.linalg.norm(A @ x_t - b) ** 2 / 2 + lambda_reg * np.linalg.norm(x_t, ord=1))
    
    for i in range(T):
        # Compute the sugradient delta
        delta = lassoSubgrad(A, b, lambda_reg, x_t)
        
        # Update estimate of x
        x_t = x_t - alpha * delta
        
        # Compute objective value at current estimate of x
        objhist.append(np.linalg.norm(A @ x_t - b) ** 2 / 2 + lambda_reg * np.linalg.norm(x_t, ord=1))
    
    return x_t, objhist

# Problem 5

# Problem 6

## Problem 6(a)

If $\mathbf{x^*}$ is a solution to $argmin_{\mathbf{x}} f(\mathbf{x})+  λ\|\mathbf{x}\|_1$, from Fermat’s optimality condition we know,

$0 \in ∇ f(\mathbf{x^*}) + λsign(\mathbf{x^*})$ which we can write as,

$ ∇ f(\mathbf{x^*}) + λsign(\mathbf{x^*}) = 0$

==> $ ∇ f(\mathbf{x^*}) = -λsign(\mathbf{x^*})$

==> $ \|∇ f(\mathbf{x^*})\|_2^2 = λ^2\|sign(\mathbf{x^*})\|_2^2 \hspace{1in} $(by sqaring 2-norm in both sides)

==> $ C^2 > λ^2\|sign(\mathbf{x^*})\|_2^2 \hspace{1in}$    (as $\|∇ f(\mathbf{x^*})\|_2 < C$)

==> $ C^2 > λ^2 n \hspace{1in}$ (where $n$ is the number of non-zero entries in $\mathbf{x^*}$)

==> $ n <  \frac{C^2}{λ^2}$

Now from the above derivation of number of non-zerp entries in $\mathbf{x^*}$, we can see that by increasing $λ$ while using $l1$ regularization, number of non-zero entries in $\mathbf{x^*}$ will be decreased proportionally which ensures that $\mathbf{x^*}$ is increasingly sparse.

## Problem 6(b)